# BoTorch

BoTorch builds on top of GPytorch and provides additional functionality for Bayesian optimization.

Fit GP kernel hyperparameters to data.

In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood

# Double precision is highly recommended for GPs.
# See https://github.com/pytorch/botorch/discussions/1444
train_X = torch.rand(10, 2, dtype=torch.double)
Y = 1 - (train_X - 0.5).norm(dim=-1, keepdim=True)  # explicit output dimension
Y += 0.1 * torch.rand_like(Y)
train_Y = (Y - Y.mean()) / Y.std()

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

/home/matthijs/bsc/BachelorProject/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

Construct an acquisition function:

In [2]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

In [3]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

print(candidate)

tensor([[0.5782, 0.2912]])


In [7]:
print(train_X.shape)
distr = gp.posterior(train_X)
print(distr.mean)
print(distr.variance)


torch.Size([10, 2])
tensor([[-0.6332],
        [ 0.0357],
        [-0.2415],
        [ 0.3599],
        [ 1.4153],
        [-0.0082],
        [ 0.3038],
        [ 0.0386],
        [-0.8361],
        [-0.4343]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
tensor([[0.2545],
        [0.1602],
        [0.2103],
        [0.2171],
        [0.3221],
        [0.2819],
        [0.2283],
        [0.2747],
        [0.2919],
        [0.2796]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)


In [26]:
print(train_X)

means = gp.posterior(train_X).mean

print(means)

print(torch.argmax(means, dim=0))

tensor([[0.9573, 0.7599],
        [0.3754, 0.9189],
        [0.4446, 0.9791],
        [0.8614, 0.5956],
        [0.5235, 0.2322],
        [0.9797, 0.3877],
        [0.2849, 0.8359],
        [0.7347, 0.8691],
        [0.0226, 0.1927],
        [0.0600, 0.4838]], dtype=torch.float64)
tensor([[-0.6332],
        [ 0.0357],
        [-0.2415],
        [ 0.3599],
        [ 1.4153],
        [-0.0082],
        [ 0.3038],
        [ 0.0386],
        [-0.8361],
        [-0.4343]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
tensor([4])
